# Data wrangling - transforming the data into a more suitable format

In [1]:
# necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The first thing to do is to load the hourly data that was pulled using the scripts in the data fetching section.

In [2]:
ts = pd.read_csv("hourly_dropped.csv", parse_dates=['timestamp'])
ts.head()

,timestamp,locationName,value
0,2017-01-01 00:00:00,1512 Malmin raitti 3,0.04
1,2017-01-01 01:00:00,1512 Malmin raitti 3,0.04
2,2017-01-01 02:00:00,1512 Malmin raitti 3,0.04
3,2017-01-01 03:00:00,1512 Malmin raitti 3,0.04
4,2017-01-01 04:00:00,1512 Malmin raitti 3,0.04


Next, we clean the data and restructure it into a more suitable format.

* Group the `ts` dataframe by the locations since our data contain electricity demand records from multiple locations.

* For each location, we create a dataframe. The index of each of these dataframes are the timestamps of the data (from 2017-January-01 to 2020-January-01). These dataframes are stored in the list `locs`.

* For each dataframe, we delete the duplicated records if there are any.

In [3]:
groups = ts.groupby('locationName')

# divide into different dataframes for each location
locs = [groups.get_group(df).set_index('timestamp').value for df in groups.groups]

# remove duplicated rows
locs = [df[~df.index.duplicated(keep='first')] for df in locs]

Next, we further clean the data. As there may be locations whose data are not in hourly resolution, we need to reformat those dataframes so that all of them are in the hourly resolution.

In [4]:
# create a range of timestamps covering the time range of the data.
all_dates = pd.date_range(start='2017-01-01', end='2020-01-01', freq='1H')

for idx, loc in enumerate(locs):
    if len(loc) > len(all_dates): # are there more records (timestamps) than the default dataframe
        locs[idx] = locs[idx].resample('1H').sum()

Next, we combine all dataframes in `locs` into 1 big dataframe `df`, in which each column is a location.

In [5]:
df = pd.concat(locs, join='outer', axis=1, ignore_index=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,571,572,573,574,575,576,577,578,579,580
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,82.92,36.35,14.25,45.92,34.63,11.47,2.80,43.56,0.04,8.80,...,22.46,49.56,43.48,42.4,21.15,200.0,108.0,25.91,NaN,69.0
2017-01-01 01:00:00,83.91,41.11,13.00,44.80,33.18,11.60,2.76,43.52,0.04,8.56,...,22.32,49.56,43.79,42.4,21.12,100.0,112.0,28.63,NaN,43.0
2017-01-01 02:00:00,83.31,40.55,9.45,43.68,32.55,11.46,2.76,43.87,0.04,8.80,...,22.35,50.04,43.67,42.0,21.66,100.0,112.0,26.50,NaN,42.0
2017-01-01 03:00:00,84.88,41.33,9.35,45.44,32.54,11.60,2.76,44.10,0.04,8.72,...,22.40,49.32,44.34,41.6,21.25,200.0,112.0,26.89,NaN,42.0
2017-01-01 04:00:00,86.23,40.76,9.40,52.64,32.47,11.55,4.12,43.98,0.04,8.72,...,22.40,49.68,43.55,42.0,20.99,200.0,120.0,30.83,NaN,47.0


Now, we can already spot that there are NaN entries in the dataframe. In other words, for those locations at those timestamps, there was no available data. Although it may be arbitrary, we decided to only take the columns (locations) in which there are less than 100 missing entries.

In [6]:
df2 = df.loc[:,df.isna().sum() < 100].copy()
df2.head()

,1,2,4,5,6,7,9,10,11,12,...,557,560,571,572,573,574,575,576,577,580
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,36.35,14.25,34.63,11.47,2.80,43.56,8.80,6.93,2.21,11.52,...,43.44,129.45,22.46,49.56,43.48,42.4,21.15,200.0,108.0,69.0
2017-01-01 01:00:00,41.11,13.00,33.18,11.60,2.76,43.52,8.56,6.89,2.26,11.58,...,43.44,129.44,22.32,49.56,43.79,42.4,21.12,100.0,112.0,43.0
2017-01-01 02:00:00,40.55,9.45,32.55,11.46,2.76,43.87,8.80,7.18,2.20,11.58,...,43.44,129.45,22.35,50.04,43.67,42.0,21.66,100.0,112.0,42.0
2017-01-01 03:00:00,41.33,9.35,32.54,11.60,2.76,44.10,8.72,6.83,2.27,11.64,...,44.04,129.45,22.40,49.32,44.34,41.6,21.25,200.0,112.0,42.0
2017-01-01 04:00:00,40.76,9.40,32.47,11.55,4.12,43.98,8.72,6.98,2.27,11.64,...,44.40,129.44,22.40,49.68,43.55,42.0,20.99,200.0,120.0,47.0


However, our data might still have missing timestamps, which is why we insert rows whose index are the missing timestamps.

In [7]:
for idx in all_dates.difference(df2.index):
    df2.loc[idx] = pd.Series(dtype='float64')

As one solution to the missing entries problem, we decided to interpolate the data linearly, that is, for each column (location), the missing entries are filled with values that follow a linear trend to other entries. There are other interpolation methods as well, but we decided to go with this method. This interpolation should not affect the inference much as for each column there are only at most 100 missing entries, and the dataframe has more than 25000 rows.

After the interpolation step, we sum over all columns (locations) as we only wish to predict the hourly "total" electricity demand.

In [8]:
df3 = df2.sort_index().interpolate().sum(axis=1).rename('kWh')
df3.head()

timestamp
2017-01-01 00:00:00    7985.77
2017-01-01 01:00:00    7780.45
2017-01-01 02:00:00    7676.35
2017-01-01 03:00:00    7722.80
2017-01-01 04:00:00    7716.57
Name: kWh, dtype: float64

As a final step in this section, we save the aggregated data.

In [9]:
file_name = 'hourly_total.csv'
df3.to_csv(file_name)